In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
import sys

/home/user/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_npz = '/media/user/Hard_Disk/Dataset/child_accident_2/batch_start2end_score/training/'
test_npz = '/media/user/Hard_Disk/Dataset/child_accident_2/batch_start2end_score/testing/'
train_path = "/media/user/Hard_Disk/Dataset/child_accident_2/batch_start2end_score/training/"
test_path = "/media/user/Hard_Disk/Dataset/child_accident_2/batch_start2end_score/testing/"
save_path = "./model2/"

In [3]:
# build model
# parameter
def build_model():
    batch_size = 6
    learning_rate = 0.001

    # feature dimension
    n_frames = 100
    n_frames_2 = 100
    n_detection = 20
    n_image_feature = 4096
    n_action_feature = 1024

    # hidden layer
    n_image_hidden = 256
    n_action_hidden = 128
    n_att_hidden = 256
    # n_score_hidden = 32
    n_hidden = 640
    n_classes = 2
    #####################
    #       input       #
    #####################
    x_image = tf.placeholder("float",[None, n_frames, n_detection, n_image_feature])
    x_action = tf.placeholder("float",[None, n_frames, 1, n_action_feature])
    # x_score = tf.placeholder("float", [None, n_frames, 1])
    keep = tf.placeholder("float",[None])
    # y_score = tf.placeholder("float",[None, n_frames, n_classes])
#     y_label = tf.placeholder("float",[None, 1])
    y_class = tf.placeholder("float",[None, n_classes])
    #####################
    #       weight      #
    #####################
    weights = {
        'image': tf.Variable(tf.random_normal([n_image_feature,n_image_hidden], mean=0.0, stddev=0.01)),
        'object': tf.Variable(tf.random_normal([n_image_feature,n_att_hidden], mean=0.0, stddev=0.01)),
        'action': tf.Variable(tf.random_normal([n_action_feature,n_action_hidden], mean=0.0, stddev=0.01)),
    #     'score': tf.Variable(tf.random_normal([1,n_score_hidden], mean=0.0, stddev=0.01)),
        'att_w': tf.Variable(tf.random_normal([n_att_hidden, 1], mean=0.0, stddev=0.01)),
        'att_wa': tf.Variable(tf.random_normal([n_hidden, n_att_hidden], mean=0.0, stddev=0.01)),
        'att_ua': tf.Variable(tf.random_normal([n_att_hidden, n_att_hidden], mean=0.0, stddev=0.01)),
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=0.0, stddev=0.01)),
        'decoder_out': tf.Variable(tf.random_normal([n_hidden, 1], mean=0.0, stddev=0.01)),
        'stack_1to3': tf.Variable(tf.random_normal([n_hidden*2, n_hidden*2], mean=0.0, stddev=0.01)),
        'stack_1to2': tf.Variable(tf.random_normal([n_hidden*2, n_hidden*2], mean=0.0, stddev=0.01)),
        'stack_2to2': tf.Variable(tf.random_normal([n_hidden*2, n_hidden*2], mean=0.0, stddev=0.01)),
        'stack_2to3': tf.Variable(tf.random_normal([n_hidden*2, n_hidden*2], mean=0.0, stddev=0.01)),
        'stack_3to3': tf.Variable(tf.random_normal([n_hidden*2, n_hidden*2], mean=0.0, stddev=0.01)),
        'predict_class': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=0.0, stddev=0.01))
    }
    biases = {
        'image': tf.Variable(tf.random_normal([n_image_hidden], mean=0.0, stddev=0.01)),
        'object': tf.Variable(tf.random_normal([n_att_hidden], mean=0.0, stddev=0.01)),
        'action': tf.Variable(tf.random_normal([n_action_hidden], mean=0.0, stddev=0.01)),
    #     'score': tf.Variable(tf.random_normal([n_score_hidden], mean=0.0, stddev=0.01)),
        'att_ba': tf.Variable(tf.zeros([n_att_hidden])),
        'out': tf.Variable(tf.random_normal([1], mean=0.0, stddev=0.01)),
        'decoder_out': tf.Variable(tf.random_normal([1], mean=0.0, stddev=0.01)),
        'fusion_2': tf.Variable(tf.random_normal([n_hidden*2], mean=0.0, stddev=0.01)),
        'fusion_3': tf.Variable(tf.random_normal([n_hidden*2], mean=0.0, stddev=0.01)),
        'predict_class': tf.Variable(tf.random_normal([n_classes], mean=0.0, stddev=0.01))

    }

    #####################
    #        LSTM       #
    #####################
    #      Encoder      #
    #####################
    lstm_cell = tf.nn.rnn_cell.LSTMCell(n_hidden,initializer= tf.random_normal_initializer(mean=0.0,stddev=0.01),use_peepholes = True,state_is_tuple = False)
    lstm_cell_stack_2 = tf.nn.rnn_cell.LSTMCell(n_hidden,initializer= tf.random_normal_initializer(mean=0.0,stddev=0.01),use_peepholes = True,state_is_tuple = False)
    lstm_cell_stack_3 = tf.nn.rnn_cell.LSTMCell(n_hidden,initializer= tf.random_normal_initializer(mean=0.0,stddev=0.01),use_peepholes = True,state_is_tuple = False)


    lstm_cell_dropout = tf.nn.rnn_cell.DropoutWrapper(lstm_cell,output_keep_prob=1 - keep[0])
    lstm_cell_stack_2_dropout = tf.nn.rnn_cell.DropoutWrapper(lstm_cell_stack_2,output_keep_prob=1 - keep[0])
    lstm_cell_stack_3_dropout = tf.nn.rnn_cell.DropoutWrapper(lstm_cell_stack_3,output_keep_prob=1 - keep[0])



    istate = tf.zeros([batch_size, lstm_cell.state_size])
    h_prev = tf.zeros([batch_size, n_hidden])

    stack_istate_2 = tf.zeros([batch_size, lstm_cell_stack_2.state_size])
    stack_h_prev_2 = tf.zeros([batch_size, n_hidden])

    stack_istate_3 = tf.zeros([batch_size, lstm_cell_stack_3.state_size])
    stack_h_prev_3 = tf.zeros([batch_size, n_hidden])

    zeros_object = tf.to_float(tf.not_equal(tf.reduce_sum(tf.transpose(x_image[:,:,1:n_detection,:],[1,2,0,3]),3),0)) # frame x n x b
    loss = 0.0

    for i in range(n_frames):
        # x_image: (n_batch, n_frame, n_detection, n_feature): (10, 300, 20, 4096)
        X_image = tf.transpose(x_image[:,i,:,:], [1, 0, 2])   # shape:(20, 10, 4096)
        X_action = tf.transpose(x_action[:,i,:,:], [1, 0, 2]) # shape:(1, 10, 4096)

        image = tf.matmul(X_image[0,:,:],weights['image']) + biases['image']
        action = tf.matmul(X_action[0,:,:],weights['action']) + biases['action']
    #     score = tf.matmul(x_score[:,i], weights['score']) + biases['score']

        # object embedding
        n_object = tf.reshape(X_image[1:n_detection,:,:], [-1, n_image_feature])  # (19, 10, 4096) -> (190, 4096)
        n_object = tf.matmul(n_object, weights['object']) + biases['object'] # (190, 4096) * (4096, n_att_hidden) + (n_hidden) = (190, n_att_hidden)
        n_object = tf.reshape(n_object,[n_detection-1,batch_size,n_att_hidden]) # Reshape -> (19, 10, n_att_hidden)
        n_object = tf.multiply(n_object,tf.expand_dims(zeros_object[i],2))

        # object attention
        brcst_w = tf.tile(tf.expand_dims(weights['att_w'], 0), [n_detection-1,1,1])
        image_part = tf.matmul(n_object, tf.tile(tf.expand_dims(weights['att_ua'], 0), [n_detection-1,1,1])) + biases['att_ba'] # n x b x h
        e = tf.tanh(tf.matmul(h_prev,weights['att_wa'])+image_part) # n x b x h    
        alphas = tf.multiply(tf.nn.softmax(tf.reduce_sum(tf.matmul(e,brcst_w),2),0),zeros_object[i])
        # weighting sum
        attention_list = tf.multiply(tf.expand_dims(alphas,2),n_object)
        attention = tf.reduce_sum(attention_list,0) # b x h

        # concat frame & object
        fusion = tf.concat([image , attention, action], 1)

#         if i > 0 :  tf.get_variable_scope().reuse_variables()
        states_all = []
        outputs_all = []
        with tf.variable_scope("LSTM") as vs:
            outputs,istate = lstm_cell_dropout(fusion,istate)
            lstm_variables = [v for v in tf.global_variables() if v.name.startswith(vs.name)]

        with tf.variable_scope("LSTM_stack_2") as vs:
            if (i % 15 == 0):
                stack_outputs_2, stack_istate_2 = lstm_cell_stack_2_dropout(fusion, stack_istate_2)
                origin_fusion_1 = fusion
            else:
                stack_outputs_2, stack_istate_2 = lstm_cell_stack_2_dropout(origin_fusion_1, stack_istate_2)
            lstm_variables_2 = [v for v in tf.global_variables() if v.name.startswith(vs.name)]

        with tf.variable_scope("LSTM_stack_3") as vs:
            if (i % 30 == 0):
                stack_outputs_3, stack_istate_3 = lstm_cell_stack_3_dropout(fusion, stack_istate_3)
                origin_fusion_2 = fusion
            else:
                stack_outputs_3, stack_istate_3 = lstm_cell_stack_3_dropout(origin_fusion_2, stack_istate_3)
            lstm_variables_3 = [v for v in tf.global_variables() if v.name.startswith(vs.name)]

        stack_istate_3 = tf.matmul(istate,weights['stack_1to3']) + tf.matmul(stack_istate_2,weights['stack_2to3']) + tf.matmul(stack_istate_3,weights['stack_3to3']) + biases['fusion_3']
        stack_istate_2 = tf.matmul(istate,weights['stack_1to2']) + tf.matmul(stack_istate_2,weights['stack_2to2']) + biases['fusion_2']
    predict_class = tf.matmul(stack_outputs_3, weights['predict_class']) + biases['predict_class']
    class_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_class, logits=predict_class))
    loss = class_loss
    correct_prediction = tf.equal(tf.argmax(predict_class,1), tf.argmax(y_class, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     predict_frame = tf.matmul(stack_outputs_3,weights['out']) + biases['out']
#     prediction = tf.transpose(prediction)
    # loss = tf.cond(tf.less(prediction, y_label), tf.exp(prediction - y_label), tf.exp(2*(prediction - y_label)))
#     frame_loss = tf.reduce_mean(tf.square(predict_frame - y_label))
#     loss = class_loss + 0.1 * frame_loss
    
    

    #     # save prev hidden state of LSTM
    #     h_prev = outputs

    #     # FC to output
    # #     pred = tf.matmul(outputs,weights['out']) + biases['out'] # b x n_classes
    #     prediction = tf.nn.softmax(tf.matmul(outputs,weights['out']) + biases['out'])
    #     temp_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_score[:,i], logits=prediction))
    #     loss = tf.add(loss, temp_loss)
    with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss/n_frames_2)
    #     states_all.append(istate)
    #     outputs_all.append(outputs)
    return x_image, x_action, y_class, keep, loss, optimizer, predict_class, accuracy
    # #####################
    # #      Decoder      #
    # #####################
    # lstm_cell_2 = tf.nn.rnn_cell.LSTMCell(n_hidden,initializer= tf.random_normal_initializer(mean=0.0,stddev=0.01),use_peepholes = True,state_is_tuple = False)
    # lstm_cell_dropout_2 = tf.nn.rnn_cell.DropoutWrapper(lstm_cell_2,output_keep_prob=1 - keep[0])
    # istate = states_all[-1]
    # h_prev = outputs_all[-1]
    # loss = []
    # for i in range(n_frames_2):
    # #     print(i)
    # #     tf.get_variable_scope().reuse_variables()
    #     with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE) as vs:
    #         outputs,istate = lstm_cell_dropout_2(h_prev,istate)
    #         lstm_variables = [v for v in tf.global_variables() if v.name.startswith(vs.name)]
    #     # save prev hidden state of LSTM
    #     h_prev = outputs
    #     # FC to output
    #     pred = tf.matmul(outputs,weights['decoder_out']) + biases['decoder_out'] # b x n_classes

    #     temp_loss = tf.reduce_mean(tf.square(pred - y_score[:,i]))
    #     loss = tf.add(loss, temp_loss)
    # with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    #     optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss/n_frames_2)

In [4]:
# path = "/media/user/Hard_Disk/Dataset/child_accident_2/batch_start2end_score/testing/"
# batch_data = np.load(path+'batch_001.npz')
# batch_image_x = batch_data['data'][:,:150]
# batch_action_x = batch_data['action'][:,:150]
# batch_score_y = batch_data['score'][:,150:]
# batch_score_y  = revise_score(batch_score_y)
# batch_score_y = np.expand_dims(batch_score_y, -1)

In [5]:
annotation_file = '/media/user/Hard_Disk/Dataset/child_accident_2/annotation/accident_frame.txt'

In [6]:
w = open(annotation_file, "r")
ann = w.read()
annotation_data = []
for i in ann.split("\n"):
    b = i.split(" ")
    if (len(b) > 1):
        annotation_data.append(b[1])
annotation_data = np.array(annotation_data).astype("int32")

In [7]:
def revise_score(data):
    label = []
    scope1 = np.array([1,0])
    scope2 = np.array([0,1])
    for i in range(data.shape[0]):
        isAccident = False
        for j in range(data.shape[1]):
            if (data[i][j][1] == 1):
                if (j>87 and j < 117):
                    label.append(scope2)
                    isAccident = True
                    break
        if (isAccident == False):
            label.append(scope1)
    return np.array(label)

In [8]:
def build_data(data):
    batch_image_x = []
    batch_action_x = [] 
    batch_score_y = []
    for i in range(6):
        if (data['label'][i][0] == 1):
            batch_action_x.append(data['action'][i][50:150])
            batch_image_x.append(data['data'][i][50:150])
            batch_score_y.append([1,0])
        else:
            start = annotation_data[int(data['id'][i])-1]
            batch_action_x.append(data['action'][i][start-100:start])
            batch_image_x.append(data['data'][i][start-100:start])
            batch_score_y.append([0,1])
    return np.array(batch_image_x), np.array(batch_action_x), np.array(batch_score_y)

In [9]:
def test_all(sess, num, path, x_image, x_action, y_class, keep, loss, optimizer, predict_class, accuracy):
    total_loss = 0.0
    batch_size = 6
    epoch_loss = []
    epoch_acc = []
    for batch in range(1,num+1):
        file_name = '%03d' %batch
        batch_data = np.load(train_path+'batch_'+file_name+'.npz')
        batch_image_x, batch_action_x, batch_score_y = build_data(batch_data)
        _,batch_loss, predict_accident, acc = sess.run([optimizer,loss, predict_class, accuracy], feed_dict={x_image: batch_image_x, x_action: batch_action_x, y_class:batch_score_y,  keep: [0.0]})
#              = sess.run(, feed_dict={x_image: batch_image_x, x_action: batch_action_x,y_label:batch_score_y,  keep: [0.0]} )
#         print("batch:",batch," accident_pred:", predict_accident, "accident_ground:", batch_score_y)
        epoch_loss.append(batch_loss)
        epoch_acc.append(acc)
    print ("Loss:", np.mean(epoch_loss), " acc:",np.mean(epoch_acc))


In [10]:
# train 2
def train():
    n_epochs = 100
    train_num = 101
    test_num = 20
    batch_size = 6
    x_image, x_action, y_class, keep, loss, optimizer, predict_class, accuracy = build_model()
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.95)
    sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True,gpu_options=gpu_options))
    init = tf.global_variables_initializer()
    sess.run(init)
    save_path = "model 3 stack/"
#     saver = tf.train.Saver(max_to_keep=100)
    saver = tf.train.Saver()
    saver.restore(sess, "model 3 stack/model-5")
    for epoch in range(n_epochs):
        # random chose batch.npz
    #     epoch_loss = np.zeros((train_num,1),dtype = float)
        epoch_loss = []
        epoch_acc = []
        n_batchs = np.arange(1,train_num+1)
        np.random.shuffle(n_batchs)
        tStart_epoch = time.time()
        for batch in n_batchs:
            file_name = '%03d' %batch
            batch_data = np.load(train_path+'batch_'+file_name+'.npz')
            batch_image_x, batch_action_x, batch_score_y = build_data(batch_data)
#             print(batch_image_x.shape, batch_action_x.shape, batch_score_y.shape)
            
            
#             batch_image_x = batch_data['data'][:,137:237]
#             batch_action_x = batch_data['action'][:,137:237]
#             batch_score_y = batch_data['score'][:,150:]
#             batch_score_y = revise_score(batch_score_y)
#             print(batch_score_y.shape)
            _,batch_loss, predict_accident, acc = sess.run([optimizer,loss, predict_class, accuracy], feed_dict={x_image: batch_image_x, x_action: batch_action_x, y_class:batch_score_y,  keep: [0.5]})
#              = sess.run(, feed_dict={x_image: batch_image_x, x_action: batch_action_x,y_label:batch_score_y,  keep: [0.0]} )
#             print("batch:",batch," accident_pred:", predict_accident, "accident_ground:", class_label)
#             print("batch_loss",batch_loss)
            epoch_loss.append(batch_loss)
            epoch_acc.append(acc)
    
         # print one epoch
        print ("Epoch:", epoch+1, " done. Loss:", np.mean(epoch_loss), " acc:",np.mean(epoch_acc))
        tStop_epoch = time.time()
        print ("Epoch Time Cost:", round(tStop_epoch - tStart_epoch,2), "s")
        sys.stdout.flush()
        if (epoch+1) %5 == 0:
            saver.save(sess,save_path+"model", global_step = epoch+1)
            print ("Training")
            test_all(sess, train_num, train_path, x_image, x_action, y_class,  keep, loss, optimizer, predict_class, accuracy)
            print ("Testing")
            test_all(sess, test_num, test_path, x_image, x_action, y_class, keep, loss, optimizer, predict_class, accuracy)
    print ("Optimization Finished!")
    saver.save(sess, save_path+"final_model")

In [11]:
train()

INFO:tensorflow:Restoring parameters from model 3 stack/model-5
Epoch: 1  done. Loss: 0.7090596  acc: 0.5346535
Epoch Time Cost: 899.62 s
Epoch: 2  done. Loss: 0.7208937  acc: 0.49834982
Epoch Time Cost: 887.21 s
Epoch: 3  done. Loss: 0.7294372  acc: 0.51485145
Epoch Time Cost: 909.32 s
Epoch: 4  done. Loss: 0.73375255  acc: 0.51980203
Epoch Time Cost: 894.15 s
Epoch: 5  done. Loss: 0.7210968  acc: 0.52970296
Epoch Time Cost: 922.23 s
Training
Loss: 0.7157005  acc: 0.5214521
Testing
Loss: 0.69740874  acc: 0.5333333
Epoch: 6  done. Loss: 0.7386064  acc: 0.48184824
Epoch Time Cost: 914.17 s
Epoch: 7  done. Loss: 0.73421305  acc: 0.49999997
Epoch Time Cost: 899.91 s
Epoch: 8  done. Loss: 0.71865  acc: 0.53960395
Epoch Time Cost: 915.96 s
Epoch: 9  done. Loss: 0.7373067  acc: 0.5049505
Epoch Time Cost: 909.29 s
Epoch: 10  done. Loss: 0.716983  acc: 0.5181518
Epoch Time Cost: 900.56 s
Training
Loss: 0.71503156  acc: 0.51485145
Testing
Loss: 0.69720984  acc: 0.5333333
Epoch: 11  done. Loss: 

KeyboardInterrupt: 